*Copyright 2024 The Specials Authors. Licensed under the Apache License, Version 2.0 (the "License").*

_Some of the code in this file is adapted from:_

_modularml/mojo_<br>
_Copyright (c) 2023, Modular Inc._<br>
_Licensed under the Apache License v2.0 with LLVM Exceptions._

<div class="alert alert-block alert-info">This notebook requires Mojo <code>v0.7.0</code>. Make sure you have the correct Mojo version installed before running it.</div>

# The `expm1` function in Specials

In this notebook, we compare the implementation of the `expm1` function in Specials with the implementations from the Mojo standard library and NumPy.

The `expm1` function computes $e^x - 1$ in a numerically stable way. It is semantically equivalent to `exp(x) - 1`, but it is more accurate for `x` close to zero.

We chose to implement this function in Specials not because there is any issue with the corresponding function in the Mojo standard library, but rather because we are studying how to implement table-based numerical methods following the paper:

> Tang, P. T. P. (1992). Table-driven implementation of the expm1 function in IEEE floating-point arithmetic. _ACM Transactions on Mathematical Software (TOMS)_, 18(2), 211-222. [[Link](https://doi.org/10.1145/146847.146928)]

In terms of maximum relative error, our experiments show that the implementation in Specials is as accurate as the evaluated alternatives. Regarding the average relative error, the implementation in Specials can be at least 12.9 times more accurate.

For mean execution time, the results indicated that the implementation in Specials is about 1.07 to 3.1 times faster than the Mojo standard library implementation and about 2.7 to 13.7 times faster than the NumPy implementation.

Before delving into the experiments, let’s explore how we can use the implementation in Specials through a simple example:

In [1]:
import specials

let exp_0 = specials.exp(Float64(0))
print("exp(0) =", exp_0)

let exp_0_minus_1 = specials.expm1(Float64(0))
print("exp(0) - 1.0 =", exp_0_minus_1)


exp(0) = 1.0
exp(0) - 1.0 = 0.0


## 1. Experimental settings

In this section, we outline the experimental settings. From the definition of domains and precision considerations to the metrics used for accuracy and computational performance evaluation, these settings lay the foundation for an objective assessment.

### 1.1. Domains

We uniformly sample 250,000 values for `expm1` argument from 5 intervals of the form $[a_i, b_i]$, referred to as _domains_, where $a_i$ and $b_i$ are the minimum and maximum values of each domain.

In the experiments, we work with single- and double-precision floating-point values (`float32` and `float64`, respectively).

### 1.2. Evaluation Metrics

To measure the accuracy of the function implementations, we use the _relative error_. Let $\hat{x}$ be an approximation of the real number $x$. The relative error $E_{\text{rel}}(\hat{x})$ is given by:

$$E_{\text{rel}}(\hat{x}) = \frac{|x - \hat{x}|}{|x|}.$$

Given the argument, the exact but unknown value of each function, represented in the formula above by the real number $x$, is computed with high precision using the Python library [`mpmath`](https://mpmath.org/).

To compare different implementations in terms of accuracy, we calculate the maximum and mean values of the relative error for each combination of implementation and domain. Lower values indicate higher accuracy.

For quantifying computational performance, we measure the _execution time_: in Mojo, using the `benchmark` module, and in Python, by defining a function based on the `timeit` module.

To compare different implementations in terms of computational performance, we calculate the mean execution time for each combination of implementation and domain. Smaller results indicate better performance.

### 1.3. Auxiliary Functions

In this section, we introduce the auxiliary functions essential for conducting our experiments and measuring results.

In [2]:
%%python
from importlib.util import find_spec
import shutil
import subprocess

fix = """
-------------------------------------------------------------------------
To fix this, follow the steps in the link below:
    https://github.com/modularml/mojo/issues/1085#issuecomment-1771403719
-------------------------------------------------------------------------
"""

def install_if_missing(name: str):
    if find_spec(name.replace("-", "_")):
        return

    print(f"The package `{name}` was not found. We will install it...")
    try:
        if shutil.which("python3"): python = "python3"
        elif shutil.which("python"): python = "python"
        else:
            raise RuntimeError("Python is not on `PATH`. " + fix)
        subprocess.check_call([python, "-m", "pip", "install", name])
    except:
        raise ImportError(f"The package `{name}` was not found. " + fix)

install_if_missing("mpmath")
install_if_missing("numpy")
install_if_missing("tabulate")


In [3]:
%%python
import mpmath as mp
import numpy as np


def py_numpy_expm1(x):
    """Computes `exp(x) - 1` for all elements in the given array using NumPy."""
    return np.expm1(x)


def py_mpmath_expm1(x):
    """Computes `exp(x) - 1` for all elements in the given array using mpmath."""
    def _mp_expm1_impl(a):
        with mp.workdps(350):
            res = mp.expm1(mp.mpf(a))
        return res

    dtype = np.result_type(x)
    return np.frompyfunc(_mp_expm1_impl, 1, 1)(x).astype(dtype)


In [4]:
import math

from python import Python
from python.object import PythonObject

from specials._internal.tensor import (
    elementwise,
    random_uniform,
    run_benchmark,
    tensor_to_numpy_array,
    UnaryOperator,
)

Python.add_to_path(".")


fn solution_report[
    solution_name: StringLiteral,
    func: UnaryOperator,
    dtype: DType,
    simd_width: Int = simdwidthof[dtype](),
](x: Tensor[dtype], truth: PythonObject) raises -> PythonObject:
    """Computes the evaluation metrics for a given numerical solution in Mojo."""
    let builtins = Python.import_module("builtins")
    let np = Python.import_module("numpy")
    let numerics_testing = Python.import_module("specials._internal.numerics_testing")

    let result = elementwise[func](x)
    let msecs = run_benchmark[func](x).mean("ms")
    let relerr = numerics_testing.py_relative_error(
        tensor_to_numpy_array(result), truth
    )

    let report = builtins.list()
    _ = report.append(solution_name)
    _ = report.append(np.max(relerr))
    _ = report.append(np.mean(relerr))
    _ = report.append(msecs)

    return report


In [5]:
%%python
from timeit import timeit

from specials._internal import numerics_testing


def py_benchmark(func, *args):
    """Computes the average execution time of a Python function."""
    # Warmup phase
    _ = timeit(lambda: func(*args), number=2)

    msecs = 1000 * timeit(lambda: func(*args), number=100) / 100
    return msecs


def py_solution_report(solution_name, func, x_arr, truth):
    """Computes the evaluation metrics for a given numerical solution in Python."""
    result = func(x_arr)
    msecs = py_benchmark(func, x_arr)
    relerr = numerics_testing.py_relative_error(result, truth)

    return [solution_name, np.max(relerr), np.mean(relerr), msecs]


In [6]:
%%python
from tabulate import tabulate, SEPARATING_LINE


def py_format_domain_name(domain_name):
    """Formats the domain name for printing."""
    values = [float(n) for n in domain_name.split(",")]
    formatted = []

    for value in values:
        if value == int(value):
            if np.abs(value) > 10e3:
                formatted.append(f"{int(value):.0e}")
            else:
                formatted.append(f"{int(value)}")
        else:
            if np.abs(value) < 0.001:
                formatted.append(f"{value:.0e}")
            elif np.log10(np.abs(value)) >= 3:
                formatted.append(f"{value:.1e}")
            else:
                formatted.append(f"{value:.1f}")

    return f"{formatted[0]},{formatted[1]}"


def py_print_table(
    data, domain_names, num_solutions, experiment_name
):
    """Prints the evaluation metrics for all numerical solutions."""
    headers = [
        "\nDomain",
        "\nSolution",
        "Maximum\nRelative Error",
        "Mean\nRelative Error",
        "Mean Execution Time\n(in milliseconds)",
    ]

    # Insert domain names
    current_domain = 0
    for i, report in enumerate(data):
        if i % num_solutions == 0:
            domain_name = py_format_domain_name(domain_names[current_domain])
            data[i].insert(0, domain_name)
            current_domain += 1
        else:
            data[i].insert(0, "")

    # Insert horizontal lines between domains
    for index in range(num_solutions, len(data) + num_solutions, num_solutions + 1):
        data.insert(index, SEPARATING_LINE)

    print(f"\nExperiment: {experiment_name}\n")

    floatfmt = (".2e", ".2e", ".2e", ".2e", ".3f")
    table = tabulate(data, headers, tablefmt="simple", floatfmt=floatfmt)

    print(table)


In [7]:
import random

from utils.static_tuple import StaticTuple


fn run_experiment[
    dtype: DType,
    num_domains: Int,
](
    name: StringLiteral,
    min_values: StaticTuple[num_domains, FloatLiteral],
    max_values: StaticTuple[num_domains, FloatLiteral],
    num_samples: Int,
    truth_func: PythonObject,
    numpy_func: PythonObject,
    report_func: PythonObject,
    print_func: PythonObject,
) raises:
    """Runs the experiment."""
    let builtins = Python.import_module("builtins")

    random.seed(42)

    let domain_names = builtins.list()
    let data = builtins.list()

    print("Running the experiment. This may take a while...\n")

    for i in range(len(max_values)):
        let min_value = min_values[i]
        let max_value = max_values[i]

        _ = domain_names.append(String("") + str(min_value) + "," + str(max_value))

        let a = random_uniform[dtype](min_value, max_value, num_samples)
        let a_arr = tensor_to_numpy_array(a)

        # mpmath
        let truth = truth_func(a_arr)

        # Specials
        let specials_report = solution_report["Specials", specials.expm1, dtype](
            a, truth
        )
        _ = data.append(specials_report)

        # Mojo
        let mojo_report = solution_report["Mojo", math.expm1, dtype](a, truth)
        _ = data.append(mojo_report)

        # NumPy
        let numpy_report = report_func("NumPy", numpy_func, a_arr, truth)
        _ = data.append(numpy_report)

    _ = print_func(data, domain_names, 3, String(name) + " (" + str(dtype) + ")")


## 2. Experiment results

In this section, we delve into the results of our experiments, comparing the implementation of `expm1` in Specials with its counterparts in Mojo standard library and NumPy.


We present the findings across two precision settings: `float32` and `float64`. Each experiment explores accuracy and computational performance across distinct domains.

In [8]:
let sml_f32 = 1e-7

run_experiment[DType.float32, num_domains=5](
    name="Expm1 Function",
    min_values=StaticTuple[5, FloatLiteral](-100 * sml_f32, -1.0, -10, -30.0, -85.0),
    max_values=StaticTuple[5, FloatLiteral](100 * sml_f32, 1.0, 10.0, 30.0, 85.0),
    num_samples=250_000,
    truth_func=py_mpmath_expm1,
    numpy_func=py_numpy_expm1,
    report_func=py_solution_report,
    print_func=py_print_table,
)


Running the experiment. This may take a while...


Experiment: Expm1 Function (float32)

                                   Maximum              Mean    Mean Execution Time
Domain        Solution      Relative Error    Relative Error      (in milliseconds)
------------  ----------  ----------------  ----------------  ---------------------
-1e-05,1e-05  Specials            0.00e+00          0.00e+00                  0.089
              Mojo                7.15e-08          2.86e-13                  0.223
              NumPy               7.15e-08          2.86e-13                  0.693
------------  ----------  ----------------  ----------------  ---------------------
-1,1          Specials            1.19e-07          5.76e-10                  0.200
              Mojo                1.19e-07          7.66e-09                  0.433
              NumPy               1.19e-07          7.66e-09                  1.810
------------  ----------  ----------------  ----------------  ---------

In [9]:
let sml_f64 = 2e-16

run_experiment[DType.float64, num_domains=5](
    name="Expm1 Function",
    min_values=StaticTuple[5, FloatLiteral](-100 * sml_f64, -1.0, -10, -30.0, -85.0),
    max_values=StaticTuple[5, FloatLiteral](100 * sml_f64, 1.0, 10.0, 30.0, 85.0),
    num_samples=250_000,
    truth_func=py_mpmath_expm1,
    numpy_func=py_numpy_expm1,
    report_func=py_solution_report,
    print_func=py_print_table,
)


Running the experiment. This may take a while...


Experiment: Expm1 Function (float64)

                                   Maximum              Mean    Mean Execution Time
Domain        Solution      Relative Error    Relative Error      (in milliseconds)
------------  ----------  ----------------  ----------------  ---------------------
-2e-14,2e-14  Specials            0.00e+00          0.00e+00                  0.243
              Mojo                0.00e+00          0.00e+00                  0.261
              NumPy               0.00e+00          0.00e+00                  0.659
------------  ----------  ----------------  ----------------  ---------------------
-1,1          Specials            2.22e-16          1.11e-18                  0.487
              Mojo                2.22e-16          1.44e-17                  0.527
              NumPy               2.22e-16          1.44e-17                  1.955
------------  ----------  ----------------  ----------------  ---------

### 2.3. Comments

- In terms of maximum relative error, our experiments showed that the implementation in Specials is as accurate as the evaluated alternatives.

- Regarding the average relative error, the implementation in Specials can be at least 12.9 times more accurate.

- For mean execution time, the results indicated that the implementation in Specials is about 1.07 to 3.1 times faster than the Mojo standard library implementation and about 2.7 to 13.7 times faster than the NumPy implementation.

- The results underscore Specials' ability to provide superior accuracy without compromising computational efficiency.

## Appendix A: System information

Below, information about the system used to run the experiment.

In [10]:
%%python

subprocess.run(["modular", "-v"])
subprocess.run(["mojo", "-v"])


modular 0.4.1 (2d8afe15)
mojo 0.7.0 (af002202)


In [11]:
from sys.info import (
    os_is_linux,
    os_is_windows,
    os_is_macos,
    has_sse4,
    has_avx,
    has_avx2,
    has_avx512f,
    has_vnni,
    has_neon,
    is_apple_m1,
    has_intel_amx,
    num_physical_cores,
    _current_target,
    _current_cpu,
    _triple_attr,
)

let os: StringLiteral
if os_is_linux():
    os = "linux"
elif os_is_macos():
    os = "macOS"
else:
    os = "windows"

let cpu = String(_current_cpu())
let arch = String(_triple_attr())

var cpu_features = String("")
if has_sse4():
    cpu_features += " sse4"
if has_avx():
    cpu_features += " avx"
if has_avx2():
    cpu_features += " avx2"
if has_avx512f():
    cpu_features += " avx512f"
if has_vnni():
    if has_avx512f():
        cpu_features += " avx512_vnni"
    else:
        cpu_features += " avx_vnni"
if has_intel_amx():
    cpu_features += " intel_amx"
if has_neon():
    cpu_features += " neon"
if is_apple_m1():
    cpu_features += " apple_m1"

if len(cpu_features) > 0:
    cpu_features = cpu_features[1:]

print("System Information")
print("    OS          :", os)
print("    CPU         :", cpu)
print("    Arch        :", arch)
print("    Num Cores   :", num_physical_cores())
print("    CPU Features:", cpu_features)


System Information
    OS          : linux
    CPU         : alderlake
    Arch        : x86_64-unknown-linux-gnu
    Num Cores   : 8
    CPU Features: sse4 avx avx2 avx_vnni


In [12]:
%%python
import pkg_resources
import sys

def get_version(package):
    """Returns the version of a Python package."""
    return pkg_resources.get_distribution(package).version

print("mpmath version:", mp.__version__)
print("NumPy version:", np.__version__)
print("Python version:", sys.version)
print("Tabulate version:", get_version("tabulate"))


mpmath version: 1.3.0
NumPy version: 1.26.0
Python version: 3.11.7 | packaged by conda-forge | (main, Dec 23 2023, 15:07:28) [GCC 12.3.0]
Tabulate version: 0.9.0
